In [39]:
import os
import random
import math

import pandas as pd
import numpy as np

In [ ]:
DATA_PATH = '/home/leelin/Downloads/2021_4_data'

config = {'data_path':'/home/leelin/Downloads/2021_4_data',
		  'output_path': '/tmp'
		 }

In [45]:
class NormalizedData(object):
	def __init__(self, config):
		'''
			Args:
				@param config: dictionary of the config
		'''
		self.en_train_data_path = os.path.join(
		    config['data_path'], 'train/train_en.tsv')
		self.tr_train_data_path = os.path.join(
		    config['data_path'], 'train/train_tr.tsv')
		self.predict_path = os.path.join(config['data_path'], 'to_predict.csv')
		self.en_doc_info_path = os.path.join(
		    config['data_path'], 'doc_info/en_list_result/')
		self.tr_doc_info_path = os.path.join(
		    config['data_path'], 'doc_info/tr_list_result/')

		self.output_path = config['output_path']

		self.part_range = 30000  # num of file num in doc_info 'part-xxxx'

	# def file_line_iter(self, file_path, is_doc_info=False):
	# 	'''
	# 		Args:
	# 			@param file_path: dir path needed while is_doc_info is True, otherwise
	# 				just normal file path
	# 	'''
	# 	if is_doc_info == True:

	# 	else:

	def gen_url2docidx(self, output_dir=None, max_line=100000):
		'''
			Args:
				@param output_name: if setted, will save DataFrame to target dir
				@param max_line: max number of rows per DataFrame
			return a DataFrame with columns = ['url','file_name','line_idx']
		'''

	def normalize_data(self, doc_info=False, predict_file=False, train_dir=False):
		'''
			Args:

		'''
		# ! 此处应该用如何替代
		replecment_pool = [
						['\x01', ' '],
						['<br>', ' ']]

		def inplace_modify(path, is_tr=False):
			# print("Normalize file: ", path)
			with open(path, 'r', encoding='utf8') as fr:
				lines = fr.readlines()
			for id in range(len(lines)):
				lines[id] = lines[id].lower()
				for key in replecment_pool:
					lines[id] = lines[id].replace(key[0], key[1])
				if is_tr:
					lines[id] = lines[id].replace('ı', 'i') \
                               .replace('ğ', 'g') \
                               .replace('ç', 'c') \
                               .replace('ö', 'o') \
                               .replace('ş', 's') \
                               .replace('ü', 'u')
				# print(lines[id])
			with open(path, 'w', encoding='utf8') as fr:
				fr.writelines(lines)

		if doc_info:
			for part_num in range(self.part_range):
				inplace_modify(self.en_doc_info_path+'part-%05d' % part_num)
				inplace_modify(self.tr_doc_info_path+'part-%05d' % part_num, is_tr=True)
		if predict_file:
			inplace_modify(self.predict_path)
		if train_dir:
			inplace_modify(self.en_train_data_path)
			inplace_modify(self.tr_train_data_path, is_tr=True)

	def train_test_split(*arrays, test_size=0.33, eval_size=None, shuffle=True, group_size=1):
		'''
			Splite the iterable data on index shape[0]

			Args:
				@param *arrays: pandans DataFrame or numpy array
				@param test_size: test data set size, float
				@param shuffle: use shuffle or not
				@param group_size: mulitple lines as a group (DIGIX-predict be top100)
			Return:
				(arrays[i]_train, arrays[i]_test, ...)
		'''
		# TODO: add gen_eval data
		result = []

		for array in arrays:
			idx = [(i, i+group_size) for i in range(0, len(array), group_size)]
			if shuffle:
				random.shuffle(idx)
			splite_idx = math.floor(len(idx) * test_size)

			if isinstance(array, pd.DataFrame):
				train_set = pd.DataFrame()
				test_set = pd.DataFrame()
				for idx_range in idx[:splite_idx]:  # test
					test_set = test_set.append(
					    array.iloc[idx_range[0]:idx_range[1], :], ignore_index=True)
				for idx_range in idx[splite_idx:]:  # train
					train_set = train_set.append(
					    array.iloc[idx_range[0]:idx_range[1], :], ignore_index=True)
				result.append(train_set)
				result.append(test_set)

			# TODO: append method of numpy array
			if isinstance(array, np.ndarray):
				train_set, test_set = [], []
				for idx_range in idx[:splite_idx]:
					test_set.append(array[idx_range[0]:idx_range[1], ])
				for idx_range in idx[splite_idx:]:
					train_set.append(array[idx_range[0]: idx_range[1], ])
				result.append(train_set)
				result.append(test_set)

		return tuple(result)


    def get_sample(self, queries, query, ranking, sampled_queries, qids):
		'''
			Get context base on (qid, query, ranking) and return as list
			Args:
				@param queries: dataFrame stored predict_data
				@param query: query part of target context
				@param ranking: ranking in query group
				@param sampled_queries: 
				@param pids: pid vcorresponding the sampled_queries

			Return:
				Sampled line's token list
		'''
        link_index = queries[(queries.loc[:, 'query'] == query) & (
            queries.ranking == ranking)].reset_index().at[0, 'link_index'][1:-1]
        qid = qids[sampled_queries.index(query)]
        part = link_index.split(',')[0] # name of part_xxxx
        row = int(link_index.split(',')[1]) # line num in file

        if qid[0:2] == 'en':
            with open('data/en_list_result/part-%s' % part.zfill(5), 'r') as fp:
                line = fp.readlines()[row]
        elif qid[0:2] == 'tr':
            with open('data/tr_list_result/part-%s' % part.zfill(5), 'r') as fp:
                line = fp.readlines()[row]
        else:
            print("the name of qid Error\n")
            line = "Error"

		parts = line.split('\x01')
        title = parts[1] * 20  # title has greater weight
        content = parts[2]
        url = parts[0]
        sample = title + content + url
        return sample

	
    def neg_samples(self, queries, n_tasks):
        """
        随机负采样多个样本
		Args:
        	@param queries: (pd.DataFrame) all of the queries data
        	@param n_tasks: set manualy 

        return: [], [[]]
        """
        # adding
        sampled_content = []
        qids = random.sample(set(queries.qid), n_tasks)
        sampled_queries = [queries[(queries.qid == qid) & (queries.ranking == 0)
				].reset_index().at[0, 'query'] for qid in qids]

        for query in sampled_queries:
            # positive sample
            pos_sample = self.get_sample(
                queries, query, 0, sampled_queries, qids)
            # negative sample
            neg_sample = self.get_sample(
                queries, query, 10, sampled_queries, qids)

            sampled_content.append([pos_sample, neg_sample])
        assert len(sampled_queries) == len(sampled_content)
        return sampled_queries, sampled_content



In [ ]:
data = NormalizedData({'data_path': DATA_PATH})
data.normalize_data(doc_info=True, predict_file=True, train_dir=True)


In [ ]:
def my_iter(objs):
	for obj in objs:
		cnt = 0
		for item in obj:
			if cnt >= len(obj):
				break
			yield item
			cnt += 1


In [ ]:
for i in my_iter([['1', '2'], ['a', 'b']]):
	print(i)

In [44]:
x = np.arange(27).reshape((3, 3, 3))
print(x[0:2, ])
print(isinstance(x, np.ndarray))

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]]
True
